#### Capital Bikeshare Data Collection and Analysis
The objective of this data collection project is to gather real-time bike station data from the Capital Bikeshare API, store it in a MySQL database, and use the data to create Power BI dashboards for analysis and visualization. These dashboards will provide insights into bike availability, station usage patterns, and system performance, helping stakeholders make data-driven decisions to improve service efficiency and user experience. 🚲📊

#### What is Capital Bikeshare?

Capital Bikeshare is a bike-sharing system serving the Washington, D.C. metropolitan area, including parts of Maryland and Virginia. Launched in 2010, it allows users to rent bikes from docking stations and return them to any other station in the network. The system operates on a membership or pay-per-ride basis, making it a convenient and eco-friendly transportation option for residents and tourists.

#### Bike Station Status Data Frame

Collects real-time bike station status data from the Capital Bikeshare API and stores it in a MySQL database. The dataset includes details such as the number of available bikes, docks, and scooters at each station, along with rental and return status indicators. This data is useful for monitoring station availability and optimizing bike-sharing operations.

In [1]:
import requests
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
import logging
import os
from dotenv import load_dotenv
from typing import Optional
from urllib.parse import quote_plus

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
load_dotenv()

def fetch_station_status(url: str) -> Optional[pd.DataFrame]:
    """
    Fetches bike station status data from the given URL, processes it, and returns a DataFrame.
    
    :param url: API endpoint for station status data
    :return: Processed Pandas DataFrame or None if an error occurs
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        data = response.json()

        last_updated = datetime.fromtimestamp(data["last_updated"])
        stations_status = data["data"]["stations"]

        # Select required columns
        df = pd.DataFrame(stations_status)[
            [
                "last_reported",
                "station_id",
                "num_scooters_available",
                "num_docks_available",
                "num_bikes_available",
                "is_renting",
                "is_returning",
                "is_installed",
                "num_ebikes_available",
                "num_bikes_disabled",
                "num_docks_disabled",
                "num_scooters_unavailable",
            ]
        ]

        # Convert last_reported to datetime
        df["last_reported"] = pd.to_datetime(df["last_reported"], unit="s")

        # Handle missing values and ensure integer types
        df["num_scooters_available"] = df["num_scooters_available"].fillna(0).astype(int)
        df["num_scooters_unavailable"] = df["num_scooters_unavailable"].fillna(0).astype(int)

        # Add collection timestamp
        df["collection_time"] = last_updated

        logging.info("Successfully fetched and processed station status data.")
        return df

    except Exception as e:
        logging.error(f"Error fetching station status data: {e}")
        return None


def save_to_database(df: pd.DataFrame, table_name: str) -> None:
    """
    Saves the given DataFrame to a MySQL database.
    
    :param df: Pandas DataFrame to be saved
    :param table_name: Name of the table in the database
    """
    try:
        # Get database credentials from environment variables
        DB_USER = os.getenv("DB_USER")
        DB_PASSWORD = os.getenv("DB_PASSWORD")
        DB_HOST = os.getenv("DB_HOST")
        DB_NAME = os.getenv("DB_NAME")

        # Create database connection
        engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{quote_plus(DB_PASSWORD)}@{DB_HOST}/{DB_NAME}")

        # Save to database
        df.to_sql(table_name, con=engine, if_exists="append", index=False)

        logging.info(f"Successfully saved data to {table_name} table.")

    except Exception as e:
        logging.error(f"Error saving data to database: {e}")


if __name__ == "__main__":
    # Define API URL
    API_URL = "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/station_status.json"

    # Fetch and process data
    station_data = fetch_station_status(API_URL)

    # Save to database if data is available
    if station_data is not None:
        save_to_database(station_data, "station_status")


2025-02-10 04:45:16,125 - INFO - Successfully fetched and processed station status data.
2025-02-10 04:45:24,818 - INFO - Successfully saved data to station_status table.


#### Explanation of Above Functions

1) Function: def fetch_station_status(url: str) -> Optional[pd.DataFrame]:

- This function fetches bike station status data from an API and processes it into a Pandas DataFrame.
- url: str → The function takes a string parameter (url), which is the API endpoint.
- -> Optional[pd.DataFrame] → The function returns a pandas.DataFrame if successful, or None if an error occurs.
- Optional[pd.DataFrame] means the function might return either a DataFrame or None. This helps in error handling because None can indicate a failed API request


2) Function: save_to_database(df: pd.DataFrame, table_name: str) -> None

- Saves the processed DataFrame into a MySQL database.
- df: pd.DataFrame → This function takes a Pandas DataFrame as input, which contains the cleaned bike station data.
- table_name: str → It also takes a string parameter, which is the name of the database table where the data will be stored.
- -> None → The function does not return anything. It performs an action (saving data to a database) but has no return value.

3) The if __name__ == "__main__": Block

- This ensures that the script runs only when executed directly (not when imported as a module in another script).
- This block defines the API url, fetches the data from the API, processes it into a Pandas DataFrame, and returns it.
- If something goes wrong (e.g., network failure), it returns None

- Calls save_to_database(), which inserts the DataFrame into a MySQL table named "station_status_pyth".